## Урок 4. Система управления базами данных MongoDB в Python

**1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.** 

**2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).**


In [342]:
from pprint import pprint
from bs4 import BeautifulSoup
import requests
import hashlib
import re
from lxml import html
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from urllib.parse import urlencode, urljoin

In [343]:
!pip install pymongo

Defaulting to user installation because normal site-packages is not writeable


In [344]:
client = MongoClient('localhost', 27017)

In [345]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

In [346]:
headers = {
    'User-Agent': USER_AGENT,
}

In [347]:
db_mongo = client['vacancy']
# для полного стирания базы данных
db_mongo.drop_collection('vacancy') 
vacancy = db_mongo.vacancy

In [348]:
# Функция для заполнения базы данных
def insert_db(dict_vacancys):
    for vacancy_dict in dict_vacancys.values():
        try:
            vacancy.insert_one({
                '_id': (hashlib.sha256(vacancy_dict["Ссылка на вакансию"].encode())).hexdigest(),
                '_vacancy': vacancy_dict,
            })
        except DuplicateKeyError:
            print('Уже имеется такой элемент')
            continue

In [349]:
db = client.dict_vacancys

In [350]:
# result = parse_hh(url, headers)

In [351]:
def cheak_and_save_vacancies_in_db(vacancies):
  for vacancy in vacancies:
    if not len(list(collection_vacancies_hh_ru.find({'vacancy_link': vacancy['vacancy_link']}))):
      collection_vacancies_hh_ru.insert_one(vacancy)

In [352]:
cheak_and_save_vacancies_in_db(result)

In [353]:
result_find = list(collection_vacancies_hh_ru.find())

In [354]:
len(result_find)

829

In [355]:
def search_sallary_db(check_salary):
    salary_w_condition = list(
        vacancy.find(
            {'$or': [{"_vacancy.Предлагаемая зарплата.max": {'$gte': check_salary}},
                     {"_vacancy.Предлагаемая зарплата.min": {'$gte': check_salary}}]}
        ))
    return salary_w_condition


if __name__ == "__main__":
    search_vacancy = 'Python разработчик'
    base_url = 'https://hh.ru/search/vacancy'
    params = {
        'clusters': 'true',
        'area': 1,
        'ored_clusters': 'true',
        'enable_snippets': 'true',
        'salary': '',
        'text': search_vacancy,
        'page': '',
        'hhtmFrom': 'vacancy_search_list',
    }
    
    query = "?" + urlencode(params)
    url = urljoin(base_url, query)
    dict_vacancys = {}
    index_page = 1
    index_vacancy = 1
    query = "?" + urlencode(params)
    url = urljoin(base_url, query)
    while True:
        dict_vacancy = {
            'Наименование вакансии': None,
            'Предлагаемая зарплата': None,
            'Ссылка на вакансию': None,
            'Работодатель': None,
            'Расположение': None,
            'Сайт, откуда собрана вакансия': base_url,
            }

        response = requests.get(url, headers=headers)
        if response.ok:
            dom = BeautifulSoup(response.text, 'html.parser')
            quotes = dom.find_all('div', {'class': 'vacancy-serp-item'})
            for quote in quotes:
                quote_title = quote.find('a').getText()
                if quote.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}) is None:
                    quote_salary = 'None'
                else:
                    quote_salary = quote.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}).text
                pattern_salary = r'(\d+\s\d+)'
                matches_salary = re.findall(pattern_salary, quote_salary.replace(' ', ''))
                matches_currency = 'руб' if 'руб' in quote_salary else 'USD' if 'USD' in quote_salary else 'None'
                dict_salary = {
                    'min': None,
                    'max': None,
                    'валюта': matches_currency,
                }

                if len(matches_salary):
                    if len(matches_salary) > 1:
                        dict_salary['min'] = int(matches_salary[0].replace('\u202f', ''))
                        dict_salary['max'] = int(matches_salary[1].replace('\u202f', ''))
                    else:
                        if 'от' in quote_salary:
                            dict_salary['min'] = int(matches_salary[0].replace('\u202f', ''))
                        else:
                            dict_salary['max'] = int(matches_salary[0].replace('\u202f', ''))

                quote_href = quote.find('a').get('href')
                quote_company = quote.find('div', attrs={'class': 'vacancy-serp-item__meta-info-company'}).text
                quote_area = quote.find('div', attrs={'data-qa': 'vacancy-serp__vacancy-address'}).text

                dict_vacancy['Наименование вакансии'] = quote_title
                dict_vacancy['Предлагаемая зарплата'] = dict_salary
                dict_vacancy['Ссылка на вакансию'] = quote_href
                dict_vacancy['Работодатель'] = quote_company
                dict_vacancy['Расположение'] = quote_area
                dict_vacancys[index_vacancy] = dict_vacancy.copy()
                index_vacancy += 1

        if dom.find('a', attrs={'data-qa': 'pager-next'}):
            params['page'] = str(index_page)
            query = "?" + urlencode(params)
            url = urljoin(base_url, query)
            index_page += 1
        else:
            break
        
    insert_db(dict_vacancys)  # добавляем словарь в базу данных
    pprint(search_sallary_db(10000)) # ищем по критериям - зарплаты больше ввеженной суммы


[]
